## as of 9:45am 8/1 i stopped it at (but not including) '//wolfgun.bandcamp.com/album/road-to-jupiter'  and saved the csv to focus on updating the database

In [1]:
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import urllib
import numpy as np
import os
import time
from tqdm import tqdm
from selenium import webdriver
%matplotlib inline

In [50]:
df = pd.read_csv('/Users/shayneufeld/Dropbox/insight/pricecamp/data/sales.csv',index_col=0)
df = df.drop_duplicates()
print('There are %.0f unique records in the database now' % df.shape[0])
print('Total revenue: %.02f' % df.amount_paid_usd.sum())

There are 135872 unique records in the database now
Total revenue: 1025668.04


# define function for opening more albums

In [3]:
def reveal_albums(selenium_driver):    
    #try implementing same thing for albums
    showMore = selenium_driver.find_elements_by_css_selector('.showMore')
    #there are 2 possible showMore buttons - for shows and for albums
    #if there is a shows one, it comes first
    el = []
    if len(showMore) == 1:
        el = showMore[0]
    elif len(showMore) == 2:
        el = showMore[1]
    
    #if there is a show more button, make sure it's 'more albums'
    if el:
        title = browser.title
        text = el.find_element_by_tag_name('a').text
        while text == 'more releases...':
            el.click()
            
            if browser.title == title:
                text = el.find_element_by_tag_name('a').text
            else:
                text = browser.title
                
    
    return selenium_driver

In [4]:
def get_albums_urls(browser,page_type='album'):
    
    elems = []
    if page_type == 'album':
        elems = browser.find_elements_by_css_selector('.trackTitle')
    elif page_type == 'music':
        elems = browser.find_elements_by_css_selector('.music-grid-item')
        
    album_names,album_urls = [],[]

    for elem in elems[1:]:
        album_names.append(elem.text)
        album_urls.append(elem.find_element_by_tag_name('a').get_attribute('href'))
        #print(elem.text)
        #print(elem.find_element_by_tag_name('a').get_attribute('href'))

    browser.close()
    
    return album_names,album_urls

# get artists to scrap


In [13]:
artist_names = []
for url in urls_to_scrape:
    
    artist_name = df[df.url==url].head(1).artist_name.values
    
    if not artist_name:
        artist_name = df[df.url==url[6:]].head(1).artist_name.values

    if type(artist_name) is not str:
        artist_name = artist_name[0]
        
    artist_names.append(artist_name)
    

## determine URLs already scraped

In [20]:
folder = '/Users/shayneufeld/Dropbox/insight/pricecamp/data/albums_supporters'
files = os.listdir(folder)
urls_scraped = []
urls_0_supp = []
for file in files:
    if not file[0] == '.':
        d = pd.read_csv(os.path.join(folder,file),nrows = 1)
        if d.shape[0] > 0:
            urls_scraped.append(d.album_url.values[0])
        else:
            urls_0_supp.append(file)

# the scraper

In [32]:
cd '/Users/shayneufeld/github/pricecamp/scrapers/bandcamp_spider'

/Users/shayneufeld/GitHub/pricecamp/scrapers/bandcamp_spider


In [96]:
urls_df = pd.DataFrame(data={'urls_to_scrape':urls_to_scrape})
urls_df.to_csv('/Users/shayneufeld/Dropbox/insight/pricecamp/data/urls_to_scrape_0731_6pm.csv')

## as of 9:45am 8/1 i stopped it at (but not including) '//wolfgun.bandcamp.com/album/road-to-jupiter'  and saved the csv to focus on updating the database

In [62]:
urls_to_scrape_df = pd.read_csv('/Users/shayneufeld/Dropbox/insight/pricecamp/data/urls_to_scrape_0802.csv')
urls_to_scrape = urls_to_scrape_df.urls.values
urls_to_scrape.shape

(750,)

In [51]:
artist_names = []
for url in tqdm(urls_to_scrape):
    
    artist_name = df[df.url==url].head(1).artist_name.values
    
    if not artist_name:
        artist_name = df[df.url==url[6:]].head(1).artist_name.values

    if type(artist_name) is not str:
        artist_name = artist_name[0]
        
    artist_names.append(artist_name)

100%|██████████| 666/666 [00:09<00:00, 71.50it/s]


In [61]:
urls_scraped = pd.DataFrame(data={'url':urls_to_scrape})
urls_scraped.to_csv('/Users/shayneufeld/Dropbox/insight/pricecamp/data/urls_scraped_0802.csv')

In [64]:
error_urls = []

for artist_url,artist_name in zip(urls_to_scrape[335:],artist_names[335:]):
    if not artist_url in urls_scraped.url.values:
        '''
        get input parameters
        '''
        if (('records' in artist_url) or ('productions' in artist_url)):
            print('%s appears to be a label, not an artist' % artist_url)
        else:
            print(artist_url)
            if artist_url[0] == 'h':
                curr_url = artist_url
            elif artist_url[0] == '/':
                curr_url = 'https:' + artist_url
            else:
                print('URL BROKEN')

            #get list of albums and urls to go through for this artist
            browser = webdriver.Firefox()
            browser.get(curr_url)
            browser_title = browser.title

            #try to clicking throuhg more albums
            browser = reveal_albums(browser)

            #if it went to the music page, the title should have changed
            DO_NOT_SCRAPE = False
            if browser.title == browser_title: #then we are on the same page
                album_names,album_urls = get_albums_urls(browser,page_type='album')
            elif 'Music' in browser.title:
                album_names,album_urls = get_albums_urls(browser,page_type='music')
            else:
                DO_NOT_SCRAPE = True
                print('ERROR - page not album or music type: %s' % artist_url)

            if not DO_NOT_SCRAPE:
                if album_urls:
                    if len(album_urls) > 10:
                        album_urls = album_urls[:11]
                    for url in album_urls:
                        if not url in urls_scraped:
                            os.system("scrapy crawl bandcamp -a start_urls='%s'" % (str(url)))
                        else:
                            print('%s already scraped' % url)
                else:
                    print('ERROR - no album urls: %s' % artist_url)
                    error_urls.append(artist_url)

//toomanyzooz.bandcamp.com/album/subway-gawdz
//mygaybanjo.bandcamp.com/album/to-the-wolves
//mclars.bandcamp.com/album/the-edgar-allan-poe-ep
//acousticlub.bandcamp.com/track/vision-of-love-acoustic-guitar-karaoke
//tomppabeats.bandcamp.com/album/harbor-lp
//titanicsinclair.bandcamp.com
ERROR - no album urls: //titanicsinclair.bandcamp.com
//lemondemon.bandcamp.com/album/spirit-phone
//steveroach.bandcamp.com/album/etheric-imprints
http://music.gravitasrecordings.com/album/visions-remixed
//machinedrum.bandcamp.com/track/white-crown-feat-tosin-abasi
//nanobii.bandcamp.com/album/sunshine-express
//pan-daijing.bandcamp.com/album/lack-pan79
//shponglemusic.bandcamp.com/album/are-you-shpongled-remastered
//scattle.bandcamp.com/album/scavenger
//charlybliss.bandcamp.com/album/soft-serve
//amplifier.bandcamp.com/album/mystoria
//carbomb.bandcamp.com/album/meta
//shredddders.bandcamp.com/album/shredders
//slimhustla.bandcamp.com/track/she-broke-up-when-i-rolled-up
//freshselects.bandcamp.com